In [5]:
import os
import json
import torch
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [7]:
caption_dir = "/Users/anjonghyeon/Desktop/KU/3-2/DeepLearning/20242R0136COSE47402/FinalProject/src/class_captions.json"

with open (caption_dir, 'r') as file:
    class_captions = json.load(file)

In [10]:
data_dir = "/Users/anjonghyeon/Desktop/KU/3-2/DeepLearning/20242R0136COSE47402/FinalProject/data/test"
class_candidate = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]
text_inputs = []
for folder in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, folder)):
        folder = folder.replace('_', ' ')
        text_inputs.append(f"{folder}. {class_captions[folder]}")

In [13]:
image_paths = []
image_labels = []

for class_name in class_candidate:
    class_folder = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_name)
        image_paths.append(img_path)
        class_name = class_name.replace('_', ' ')
        image_labels.append(class_name)

print(f"test dataset size : {len(image_paths)}")

test dataset size : 25250


In [18]:
ans = 0
model.eval()

for idx, img_path in enumerate(image_paths):
    image = Image.open(img_path).convert('RGB')
    inputs = processor(text=text_inputs, images=image, return_tensors='pt', padding=True)
    outputs = model(**inputs)

    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    result = text_inputs[torch.argmax(probs).item()]
    if result.split('.')[0] == image_labels[idx]:
        ans += 1
    if idx % 100 == 0:
        print(f"# {idx} finished.")

precision = ans / len(image_paths)
precision


YES 0
result : foie gras, answer : foie gras
# 0 finished.
YES 1
result : foie gras, answer : foie gras
YES 2
result : foie gras, answer : foie gras
YES 3
result : foie gras, answer : foie gras
YES 4
result : foie gras, answer : foie gras


KeyboardInterrupt: 